In [1]:
from __future__ import absolute_import, division, print_function
import os, sys, json
current_path = os.path.abspath('.')
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)
import argparse
import collections,math
import time,datetime,pytz
import numpy as np
import pandas as pd
import pickle
import unittest

from scipy.fftpack import dct, idct
from scipy import stats
from scipy import interpolate, io
from scipy.io import loadmat
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import operator
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torchvision import datasets, transforms
from tqdm import tqdm
from torchinfo import summary
import faiss

from src.data_loader import *
from src.utils import *

from src.models.simple_cnn import *
from src.models.resnet_models import *
from src.models.dscnn import *
from train_joint import *

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

############################################
########## Plot Style Declaration ##########
# Set the style globally
# Alternatives include bmh, fivethirtyeight, ggplot,
# dark_background, seaborn-deep, etc
# plt.style.use('ggplot')
plt.style.use('seaborn-white')

plt.rcParams['font.family'] = 'times new roman'
# plt.rcParams['font.serif'] = 'Ubuntu'
# plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 15
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

##################################
########## End of Setup ##########

In [17]:
class ArgClass():
    def __init__(self):
        super(ArgClass, self).__init__()

args = ArgClass()
args.dataset="mnist"
args.model_arch = "cnn"
args.batch_size=128
args.test_batch_size=1000
args.percent=[0.8, 0.92, 0.991, 0.93]
args.alpha=5e-4
args.rho=1e-2
args.l1=False
args.l2=False
args.num_pre_epochs=1
args.num_epochs=1
args.num_re_epochs=3
args.lr=1e-3
args.adam_epsilon=1e-8
args.no_cuda=False
args.seed=1
args.save_model=False
args.shuffle=True

args.optimizer_name = 'adam'
args.lr_mode = 'multistep'
args.lr_decay = 0.1
args.lr_decay_epoch = '20,40'
args.target_lr = 1e-8
args.warmup_epochs = 0
args.warmup_lr = 1e-8
args.warmup_mode = 'linear'
args.momentum = 0.9
args.wd = 0.0001
args.gamma_wd_mult = 1.0
args.beta_wd_mult = 1.0
args.bias_wd_mult = 1.0
args.grad_clip = None
args.label_smoothing = False


args.test_fold_l = '[10]'
args.use_one_task = 'false'
args.exp_setup = ''
args.subject_idx = None
args.pretrained=False
args.best_acc = 200.0
args.mixup = False
args.mixup_alpha = 1.0
args.mixup_epoch_tail = 10
args.session = 1
args.test_vote = None

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [3]:
from torch.quantization import QuantStub, DeQuantStub
class PTCVDSCNNQuant(nn.Module):
    def __init__(self, model_name="dscnn_l", in_channels=1, num_classes=12, init_block_kernel=(10,4), pretrained=False, **kwargs):
        super(PTCVDSCNNQuant, self).__init__()
        self.model = get_dscnn(model_name=model_name.lower(),
                               in_channels=in_channels,
                               num_classes=num_classes,
                               init_block_kernel=init_block_kernel,
                               pretrained=pretrained,
                               **kwargs)
        self.quant = QuantStub()
        self.dequant = DeQuantStub()
        
    def forward(self, x):
        x = self.quant(x)
        x = self.model(x)
        x = self.dequant(x)
        return F.log_softmax(x, dim=1)


class AudioCNNQuant(nn.Module):
    def __init__(self,
                 channels=[32,32,64,64],
                 in_channels=1,
                 in_size=(32, 32),
                 num_classes=10,
                 pooling_type='avg'
                 ):
        super(AudioCNNQuant, self).__init__()

        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels,channels[0],kernel_size=(3, 3),stride=1,padding=1),
                        nn.BatchNorm2d(channels[0]),
                        nn.ReLU(inplace=True),
                    )
        self.conv2 = nn.Sequential(
                        nn.Conv2d(channels[0], channels[1], kernel_size=(3, 3), stride=1, padding=1),
                        nn.BatchNorm2d(channels[1]),
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(2, stride=2) if pooling_type == 'max' else nn.AvgPool2d(2, stride=2)
                    )
        self.conv3 = nn.Sequential(
                        nn.Conv2d(channels[1], channels[2], kernel_size=(3, 3), stride=1, padding=1),
                        nn.BatchNorm2d(channels[2]),
                        nn.ReLU(inplace=True),
                    )
        self.conv4 = nn.Sequential(
                        nn.Conv2d(channels[2], channels[3], kernel_size=(3, 3), stride=1, padding=1),
                        nn.BatchNorm2d(channels[3]),
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(2, stride=2) if pooling_type == 'max' else nn.AvgPool2d(2, stride=2)
                    )

        self.dropout = nn.Dropout(0.5)
        features_dim, seq_dim = get_feature_seq_dim(4, in_size)
        self.fc = nn.Linear(channels[3]*features_dim*seq_dim, num_classes)
        self.quant = QuantStub()
        self.dequant = DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.dropout(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.dropout(x)
        # x = x.view(x.size(0), -1)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        
        x = self.dequant(x)
        return F.log_softmax(x, dim=1)

    
class VWNetQuant(nn.Module):
    def __init__(self,
                 args,
                 in_channels=1,
                 in_size=(32, 32),
                 channels=[32, 32, 64, 64],
                 fcDims=[128, 128],
                 kernels=[(3,3)],
                 num_classes=10,
                 pooling_type='avg'
                 ):
        super(VWNetQuant, self).__init__()

        self.features = nn.Sequential()
        self.features.add_module("init_block",
             nn.Sequential(
                 nn.Conv2d(in_channels, channels[0], kernel_size=kernels[0], stride=1,padding=0),
                 nn.ReLU(inplace=True),
                 nn.MaxPool2d(2, stride=2) if pooling_type == 'max' else nn.AvgPool2d(2, stride=2))
         )

        for i, channels_per_stage in enumerate(channels):
            if i == 0:
                continue
            self.features.add_module("stage{}".format(i),
                 nn.Sequential(
                     nn.Conv2d(channels[i-1], channels[i], kernel_size=kernels[i] if len(kernels)>1 else kernels[0], stride=1,padding=0),
                     nn.ReLU(inplace=True),
                     nn.MaxPool2d(2, stride=2) if pooling_type == 'max' else nn.AvgPool2d(2, stride=2))
             )

        self.dropout = nn.Dropout(0.5)
        features_dim, seq_dim = get_feature_seq_dim_nopadding(len(channels), in_size, kernels)

        self.decoder = nn.Sequential()
        self.decoder.add_module("init_decoder",
            nn.Sequential(
                nn.Linear(channels[-1] * features_dim * seq_dim, fcDims[0]),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5))
        )

        if len(fcDims) > 1:
            for i, channels_per_stage in enumerate(fcDims):
                if i == 0:
                    continue
                self.decoder.add_module("decode_stage{}".format(i),
                    nn.Sequential(
                        nn.Linear(fcDims[i-1], fcDims[i]),
                        nn.ReLU(inplace=True),
                        nn.Dropout(0.5))
                )

        self.output = nn.Linear(fcDims[-1], num_classes)
        self.quant = QuantStub()
        self.dequant = DeQuantStub()


    def forward(self, x):
        x = self.quant(x)
        x = self.features(x)
        # print(x.size())
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        x = self.output(x)
        x = self.dequant(x)
        return F.log_softmax(x, dim=1)

    


In [4]:

mnist = VWNetQuant(args,
            in_channels=1,
            in_size=(28,28),
            channels=[8,16,32],
            fcDims=[128,256],
            kernels=[(3,3)],
            num_classes=10,
            pooling_type='max')
mnist.load_state_dict(torch.load('../data/saved_model/VWNet_mnist_adam_multistep_nomixup.pt', map_location=device))


# 90.05
args.config = DATASET_CONFIGS['cifar10']
cifar10 = PTCVDSCNNQuant(model_name='MicroNet-AD-I-ch168', in_channels=args.config["in_channels"],
                             num_classes=args.config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=False)
cifar10.load_state_dict(torch.load('../data/saved_model/MicroNet-AD-I-ch168_cifar10_adam_multistep_nomixup.pt', map_location=device))

# 94.48
args.config = DATASET_CONFIGS['svhn']
svhn = PTCVDSCNNQuant(model_name='MicroNet-AD-I-ch168', in_channels=args.config["in_channels"],
                             num_classes=args.config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=False)
svhn.load_state_dict(torch.load('../data/saved_model/MicroNet-AD-I-ch168_svhn_adam_multistep_nomixup.pt', map_location=device))

# 97.50
args.config = DATASET_CONFIGS['gtsrb-vw']
gtsrb = PTCVDSCNNQuant(model_name='MicroNet-AD-I-ch168', in_channels=args.config["in_channels"],
                             num_classes=args.config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=False)
gtsrb.load_state_dict(torch.load('../data/saved_model/MicroNet-AD-I-ch168_gtsrb-vw_adam_multistep_nomixup.pt', map_location=device))

# 78.9
config = DATASET_CONFIGS['hhar-noaug']
hhar = PTCVDSCNNQuant(model_name='MicroNet-AD-I-ch176-s41', in_channels=1,
                             num_classes=config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=args.pretrained)
hhar.load_state_dict(torch.load('../data/saved_model/MicroNet-hhar-ch176_hhar-noaug_adam_multistep_nomixup.pt', map_location=device))



# 73.0
config = DATASET_CONFIGS['stl10']
stl10 = PTCVDSCNNQuant(model_name='DS-CNN-M-s33', in_channels=config["in_channels"],
                             num_classes=config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=args.pretrained)
stl10.load_state_dict(torch.load('../data/saved_model/DS-CNN-M-s33_stl10_adam_multistep_nomixup.pt', map_location=device))

# 93.47
config = DATASET_CONFIGS['urbansound8k-LMCST-long-randfold']
urbansound = PTCVDSCNNQuant(model_name='DS-CNN-M-s33', in_channels=1,
                             num_classes=config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=args.pretrained)
urbansound.load_state_dict(torch.load('../data/saved_model/DS-CNN-M-s33_urbansound8k-LMCST-long-randfold_adam_multistep_nomixup.pt', map_location=device))


# Unseen HAR
# 91.5
config = DATASET_CONFIGS['pamap2']
pamap2 = PTCVDSCNNQuant(model_name='MicroNet-AD-I-ch168-s21', in_channels=1,
                             num_classes=config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=args.pretrained)
pamap2.load_state_dict(torch.load('../data/saved_model/MicroNet-AD-I-ch168-s21_pamap2_adam_multistep_nomixup.pt', map_location=device))

# 97.65
config = DATASET_CONFIGS['skoda']
skoda = PTCVDSCNNQuant(model_name='MicroNet-AD-I-ch192-s22', in_channels=1,
                             num_classes=config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=args.pretrained)
skoda.load_state_dict(torch.load('../data/saved_model/MicroNet-AD-I-ch192-s22_skoda_adam_multistep_nomixup.pt', map_location=device))



<All keys matched successfully>

In [5]:
# Lee's datasets
args.dataset = 'mnist'
train_loader_mnist, test_loader_mnist = get_data_loaders(args, kwargs)
args.dataset='cifar10'
train_loader_cifar10, test_loader_cifar10 = get_data_loaders(args, kwargs)
args.dataset='svhn'
train_loader_svhn, test_loader_svhn = get_data_loaders(args, kwargs)
args.dataset='gtsrb-vw'
train_loader_gtsrb, test_loader_gtsrb = get_data_loaders(args, kwargs)
# args.dataset='gscv2-mfcc10'
# train_loader_gsc, test_loader_gsc = get_data_loaders(args, kwargs)

# additional datasets to be trained 
args.dataset = 'hhar-noaug'
train_loader_hhar, test_loader_hhar = get_data_loaders(args, kwargs)
# args.dataset = 'ninapro-db2-c10'
# args.subject_idx=11
# args.exp_setup='per-subject'
# train_loader_ninadb2, test_loader_ninadb2 = get_data_loaders(args, kwargs)

# unseen datasets
# args.dataset = 'fashion-mnist'
# train_loader_fmnist, test_loader_fmnist = get_data_loaders(args, kwargs)
args.dataset = 'stl10'
train_loader_stl10, test_loader_stl10 = get_data_loaders(args, kwargs)

# args.dataset = 'emotion-5-frames25'
# train_loader_emotion, test_loader_emotion = get_data_loaders(args, kwargs)
args.dataset = 'urbansound8k-LMCST-long-randfold'
train_loader_urbansound, test_loader_urbansound = get_data_loaders(args, kwargs)

args.dataset = 'pamap2'
train_loader_pamap2, test_loader_pamap2 = get_data_loaders(args, kwargs)
args.dataset = 'skoda'
train_loader_skoda, test_loader_skoda = get_data_loaders(args, kwargs)

# args.dataset = 'ninapro-db3-c10'
# args.subject_idx=1
# train_loader_ninadb3, test_loader_ninadb3 = get_data_loaders(args, kwargs)
# args.dataset = 'ninapro-db6-c7-seq80'
# train_loader_ninadb6, test_loader_ninadb6 = get_data_loaders(args, kwargs)

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ../data-link/image/svhn/train_32x32.mat
Using downloaded and verified file: ../data-link/image/svhn/test_32x32.mat
X.shape:  (39209, 1, 32, 32)
y.shape:  (39209,)
X.shape:  (12630, 1, 32, 32)
y.shape:  (12630,)
X.shape:  (59403, 1, 20, 120)
y.shape:  (59403,)
X.shape:  (7721, 1, 20, 120)
y.shape:  (7721,)
Files already downloaded and verified
Files already downloaded and verified
X.shape:  (7859, 1, 128, 85)
y.shape:  (7859,)
X.shape:  (873, 1, 128, 85)
y.shape:  (873,)
The shape of the train dataset is 52, and the labels is 473445
The shape of the val dataset is 52, and the labels is 90814
The shape of the test dataset is 52, and the labels is 83366
X.shape:  (35263, 1, 33, 52)
y.shape:  (35263,)
X.shape:  (5209, 1, 33, 52)
y.shape:  (5209,)
The shape of the train dataset is 60, and the labels is 143825
The shape of the val dataset is 60, and the labels is 16976
The shape of

In [23]:
import torch.quantization
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    model_size = os.path.getsize("temp.p")/1e6
    print('Size (MB):', model_size)
    os.remove('temp.p')
    return model_size

def get_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    model_size = os.path.getsize("temp.p")/1e6
    os.remove('temp.p')
    return model_size

def train(args, model, device, train_loader, test_loader, optimizer):
    args.best_acc = -0.1
    for epoch in range(args.num_epochs):
        print(f'Epoch: {epoch+1:5d}')
        args.epoch = epoch + 1
        model.train()
        for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            if args.mixup and (epoch < args.num_epochs - args.mixup_epoch_tail) :
                data, target_a, target_b, lam = mixup_data(data, target, device, alpha=args.mixup_alpha)
                output = model(data)
                loss = mixup_loss(args, model, output, target_a, target_b, lam)
            else:
                output = model(data)
                loss = regularized_nll_loss(args, model, output, target)
            loss.backward()
            optimizer.step()
        train_acc = test(args, model, device, train_loader, 3)
        test_acc = test(args, model, device, test_loader, 3)
        #if train_acc > 70.:
        #    break

def test(args, model, device, test_loader, neval_batches):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for cnt, (data, target) in enumerate(test_loader):
            if args.test_vote is None:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
                pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
            elif args.test_vote == 'avg_pool_vote':
                data, target = data.to(device), target.to(device)
                tot_window_size = data.size()[3] # NCHW
                window_size = args.config['seq']
                window_stride = args.config['seq']
                for i in range(0, tot_window_size, window_stride):
                    data_temp = data[:,:,:,i*window_stride:i*window_stride+window_size] if i*window_stride+window_size <= tot_window_size else data[:,:,:,-1-window_size:-1]
                    output_temp = model(data_temp)
                    test_loss += F.nll_loss(output_temp, target, reduction='sum').item()  # sum up batch loss
                    if i == 0:
                        output = output_temp
                    else:
                        output += output_temp
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
            if cnt >= neval_batches:
                neval = (cnt+1)*data.size(0)
                test_loss /= neval
                
                print('\nTrain or Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
                test_loss, correct, neval,
                100. * correct / neval))
                return 100. * correct / neval

    test_loss /= len(test_loader.dataset)

    print('\nTrain or Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    if args.best_acc < 100. * correct / len(test_loader.dataset):
        args.best_acc = 100. * correct / len(test_loader.dataset)
        args.best_acc_epoch = args.epoch
        print(f'New Best Accuracy: ({args.best_acc:.2f}%) at Epoch: ({args.best_acc_epoch})\n')
    return 100. * correct / len(test_loader.dataset)
        
def staticQuantizePerTensor(args, rModel, train_loader, test_loader, num_calibration_batches=32,num_eval_batches=1000):
    
    myModel = copy.deepcopy(rModel.to('cpu'))
    myModel.eval()
    size_prev = get_size_of_model(myModel)

    # Specify quantization configuration
    # Start with simple min/max range estimation and per-tensor quantization of weights
    myModel.qconfig = torch.quantization.default_qconfig
    print(myModel.qconfig)
    torch.quantization.prepare(myModel, inplace=True)

    # Calibrate with the training set
    acc_prev = test(args, myModel.to(device), device, train_loader, num_calibration_batches)
    print('Post Training Quantization: Calibration done')

    # Convert to quantized model
    torch.quantization.convert(myModel, inplace=True)
    print('Post Training Quantization: Convert done')

    size_curr = get_size_of_model(myModel)

    acc_curr = test(args, myModel.to(device), device, test_loader, num_eval_batches)
    print(f'Size Prev (MB): {size_prev}, Size Current (MB): {size_curr}, Comp Rate: {size_prev/size_curr} ')
    return acc_prev, acc_curr, size_prev/size_curr

def getResult4Quant(args, rModel, train_loader, test_loader, num_calibration_batches=32,num_eval_batches=1000):
    ret_l = []
#     acc_curr_l = []
#     comp_rate_l = []
    for i in range(5):
        torch.manual_seed(i)
        ret = staticQuantizePerTensor(args,rModel,train_loader,test_loader,num_calibration_batches,num_eval_batches)
        ret_l.append(ret)
    ret_l = np.array(ret_l)
    print(ret_l)
    print(f'Accuracy: {np.mean(ret_l[:,1])}, Acc_Std: {np.std(ret_l[:,1])}, Comp Rate Avg.: {np.mean(ret_l[:,2])}, Comp Rate Std.: {np.std(ret_l[:,2])}')

    
def staticQuantizePerChannel(args, rModel, train_loader, test_loader, num_calibration_batches=32,num_eval_batches=1000):
    
    myModel = copy.deepcopy(rModel.to('cpu'))
    myModel.eval()
    size_prev = get_size_of_model(myModel)

    # Specify quantization configuration
    # Start with simple min/max range estimation and per-tensor quantization of weights
    myModel.qconfig = torch.quantization.get_default_qconfig('fbgemm')
    print(myModel.qconfig)
    torch.quantization.prepare(myModel, inplace=True)

    # Calibrate with the training set
    acc_prev = test(args, myModel.to(device), device, train_loader, num_calibration_batches)
    print('Post Training Quantization: Calibration done')

    # Convert to quantized model
    torch.quantization.convert(myModel, inplace=True)
    print('Post Training Quantization: Convert done')

    size_curr = get_size_of_model(myModel)

    acc_curr = test(args, myModel.to(device), device, test_loader, num_eval_batches)
    print(f'Size Prev (MB): {size_prev}, Size Current (MB): {size_curr}, Comp Rate: {size_prev/size_curr} ')
    return acc_prev, acc_curr, size_prev/size_curr

def getResult4QuantChannel(args, rModel, train_loader, test_loader, num_calibration_batches=32,num_eval_batches=1000):
    ret_l = []
#     acc_curr_l = []
#     comp_rate_l = []
    for i in range(10):
        torch.manual_seed(i)
        ret = staticQuantizePerChannel(args,rModel,train_loader,test_loader,num_calibration_batches,num_eval_batches)
        ret_l.append(ret)
    ret_l = np.array(ret_l)
    print(ret_l)
    print(f'Accuracy: {np.mean(ret_l[:,1])}, Acc_Std: {np.std(ret_l[:,1])}, Comp Rate Avg.: {np.mean(ret_l[:,2])}, Comp Rate Std.: {np.std(ret_l[:,2])}')
    

def qaQuantizePerChannel(args, rModel, train_loader, test_loader, num_calibration_batches=32,num_eval_batches=1000):
    
    myModel = copy.deepcopy(rModel.to(device))
    myModel.eval()
    size_prev = get_size_of_model(myModel)

    # Specify quantization configuration
    # Start with simple min/max range estimation and per-tensor quantization of weights
    # optimizer = torch.optim.SGD(myModel.parameters(), lr = 0.001)
    optimizer = torch.optim.Adam(myModel.parameters(), lr = 0.0005, betas=(0.9, 0.999),weight_decay=0.0001)
    
    myModel.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
    print(myModel.qconfig)
    torch.quantization.prepare_qat(myModel, inplace=True)

    # Calibrate with the training set
    acc_prev = test(args, myModel.to(device), device, train_loader, num_calibration_batches)
    print('Post Training Quantization: Calibration done')
    
    for nepoch in range(8):
        train(args, myModel.to(device), device, train_loader, test_loader, optimizer)
        
        if nepoch > 3:
            # Freeze quantizer parameters
            myModel.apply(torch.quantization.disable_observer)
        if nepoch > 2:
            # Freeze batch norm mean and variance estimates
            myModel.apply(torch.nn.intrinsic.qat.freeze_bn_stats)
        
        quantized_model = torch.quantization.convert(myModel.to('cpu').eval(), inplace=False)
        quantized_model.eval()
        acc_curr = test(args, quantized_model.to('cpu'), 'cpu', test_loader, 10)
        print(f'MY Current Accuracy: {acc_curr}, at Epoch: {nepoch}')
    
    # Convert to quantized model
    torch.quantization.convert(myModel.to('cpu'), inplace=True)
    print('Post Training Quantization: Convert done')

    size_curr = get_size_of_model(myModel)

    acc_curr = test(args, myModel.to('cpu'), 'cpu', test_loader, num_eval_batches)
    print(f'Size Prev (MB): {size_prev}, Size Current (MB): {size_curr}, Comp Rate: {size_prev/size_curr} ')
    return acc_prev, acc_curr, size_prev/size_curr

def getResult4qaQuantChannel(args, rModel, train_loader, test_loader, num_calibration_batches=32,num_eval_batches=1000):
    ret_l = []
#     acc_curr_l = []
#     comp_rate_l = []
    for i in range(5):
        torch.manual_seed(i+2)
        ret = qaQuantizePerChannel(args,rModel,train_loader,test_loader,num_calibration_batches,num_eval_batches)
        ret_l.append(ret)
    ret_l = np.array(ret_l)
    print(ret_l)
    print(f'Accuracy: {np.mean(ret_l[:,1])}, Acc_Std: {np.std(ret_l[:,1])}, Comp Rate Avg.: {np.mean(ret_l[:,2])}, Comp Rate Std.: {np.std(ret_l[:,2])}')
    

## QAT Results

In [10]:
# ADAM results
getResult4qaQuantChannel(args,cifar10,train_loader_cifar10,test_loader_cifar10,32,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/391 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.1851, Accuracy: 3946/4224 (93.42%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 391/391 [00:40<00:00,  9.55it/s]



Train or Test set: Average loss: 0.1857, Accuracy: 480/512 (93.75%)


...


Train or Test set: Average loss: 0.1454, Accuracy: 488/512 (95.31%)

Train or Test set: Average loss: 0.3874, Accuracy: 3570/4000 (89.25%)


  0%|          | 0/391 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.3841, Accuracy: 8953/10000 (89.53%)
New Best Accuracy: (89.53%) at Epoch: (5)

MY Current Accuracy: 89.53, at Epoch: 1
Epoch:     1


 84%|████████▍ | 328/391 [00:49<00:10,  6.17it/s]

KeyboardInterrupt: 

In [ ]:
# ADAM results
getResult4qaQuantChannel(args,svhn,train_loader_svhn,test_loader_svhn,32,1000)

In [ ]:
# ADAM results
getResult4qaQuantChannel(args,gtsrb,train_loader_gtsrb,test_loader_gtsrb,32,1000)

In [22]:
# ADAM results: LR=0.0005
# outer loop: 4 epochs
# inner loop: 1 epochs
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,64,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.8069, Accuracy: 6504/8320 (78.17%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [08:42<00:00,  1.12s/it]



Train or Test set: Average loss: 0.0261, Accuracy: 508/512 (99.22%)

Train or Test set: Average loss: 1.5259, Accuracy: 2689/4000 (67.22%)


...


Train or Test set: Average loss: 0.0440, Accuracy: 506/512 (98.83%)

Train or Test set: Average loss: 1.6083, Accuracy: 2576/4000 (64.40%)


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 1.6430, Accuracy: 5053/7721 (65.44%)
New Best Accuracy: (65.44%) at Epoch: (1)

MY Current Accuracy: 65.44489055821785, at Epoch: 2
Epoch:     1


100%|██████████| 465/465 [00:34<00:00, 13.38it/s]



Train or Test set: Average loss: 0.0518, Accuracy: 501/512 (97.85%)

Train or Test set: Average loss: 1.4339, Accuracy: 2781/4000 (69.53%)

Train or Test set: Average loss: 1.3336, Accuracy: 5633/7721 (72.96%)
New Best Accuracy: (72.96%) at Epoch: (1)

MY Current Accuracy: 72.95687087164875, at Epoch: 3
Post Training Quantization: Convert done

Train or Test set: Average loss: 1.3336, Accuracy: 5633/7721 (72.96%)
Size Prev (MB): 0.830148, Size Current (MB): 0.300795, Comp Rate: 2.7598464070213935 
[[78.17307692 70.13340241  2.75984135]
 [91.86298077 72.95687087  2.75984641]]
Accuracy: 71.54513664033156, Acc_Std: 1.4117342313171832, Comp Rate Avg.: 2.75984388091107, Comp Rate Std.: 2.5261103235862947e-06


In [20]:
# ADAM results: LR=0.0005
# outer loop: 4 epochs
# inner loop: 1 epochs
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,64,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.4772, Accuracy: 7307/8320 (87.82%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [00:23<00:00, 19.71it/s]



Train or Test set: Average loss: 0.0675, Accuracy: 500/512 (97.66%)

Train or Test set: Average loss: 1.9126, Accuracy: 2309/4000 (57.73%)


...


Train or Test set: Average loss: 0.0299, Accuracy: 506/512 (98.83%)

Train or Test set: Average loss: 1.6082, Accuracy: 2550/4000 (63.75%)


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 1.4643, Accuracy: 5397/7721 (69.90%)
New Best Accuracy: (69.90%) at Epoch: (1)

MY Current Accuracy: 69.9002719854941, at Epoch: 2
Epoch:     1


100%|██████████| 465/465 [00:23<00:00, 20.12it/s]



Train or Test set: Average loss: 0.0351, Accuracy: 505/512 (98.63%)

Train or Test set: Average loss: 1.2474, Accuracy: 2814/4000 (70.35%)

Train or Test set: Average loss: 1.1962, Accuracy: 5733/7721 (74.25%)
New Best Accuracy: (74.25%) at Epoch: (1)

MY Current Accuracy: 74.2520398912058, at Epoch: 3
Post Training Quantization: Convert done

Train or Test set: Average loss: 1.1962, Accuracy: 5733/7721 (74.25%)
Size Prev (MB): 0.830148, Size Current (MB): 0.300797, Comp Rate: 2.7598280567957794 
[[87.82451923 66.39036394  2.75991476]
 [86.91105769 64.77140267  2.75982141]
 [89.66346154 71.04002072  2.75983643]
 [90.66105769 69.34334931  2.75987141]
 [88.26923077 74.25203989  2.75982806]]
Accuracy: 69.15943530630747, Acc_Std: 3.359393813725674, Comp Rate Avg.: 2.759854412428684, Comp Rate Std.: 3.47491018200486e-05


In [18]:
# ADAM results: LR=0.0005
# outer loop: 2 epochs
# inner loop: 1 epochs
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,64,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.4772, Accuracy: 7307/8320 (87.82%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [00:23<00:00, 19.94it/s]



Train or Test set: Average loss: 0.0597, Accuracy: 500/512 (97.66%)

Train or Test set: Average loss: 1.3861, Accuracy: 2587/4000 (64.67%)


...


Train or Test set: Average loss: 0.0353, Accuracy: 504/512 (98.44%)

Train or Test set: Average loss: 2.1914, Accuracy: 2478/4000 (61.95%)


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 2.2358, Accuracy: 4971/7721 (64.38%)
New Best Accuracy: (64.38%) at Epoch: (1)

MY Current Accuracy: 64.38285196218106, at Epoch: 0
Epoch:     1


100%|██████████| 465/465 [00:23<00:00, 19.72it/s]



Train or Test set: Average loss: 0.0569, Accuracy: 505/512 (98.63%)

Train or Test set: Average loss: 1.7365, Accuracy: 2554/4000 (63.85%)

Train or Test set: Average loss: 1.5767, Accuracy: 5444/7721 (70.51%)
New Best Accuracy: (70.51%) at Epoch: (1)

MY Current Accuracy: 70.50900142468592, at Epoch: 1
Post Training Quantization: Convert done

Train or Test set: Average loss: 1.5767, Accuracy: 5444/7721 (70.51%)
Size Prev (MB): 0.830152, Size Current (MB): 0.300791, Comp Rate: 2.759896406474928 
[[87.82451923 68.15179381  2.75985306]
 [86.91105769 64.21447999  2.75984641]
 [89.66346154 62.93226266  2.75983976]
 [90.66105769 71.29905453  2.75991728]
 [88.26923077 70.50900142  2.75989641]]
Accuracy: 67.4213184820619, Acc_Std: 3.3328562008100477, Comp Rate Avg.: 2.759870582294662, Comp Rate Std.: 3.0625749227012536e-05


In [16]:
# ADAM results: LR=0.0005
# outer loop: 2 epochs
# inner loop: 5 epochs
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,64,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.4772, Accuracy: 7307/8320 (87.82%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [10:50<00:00,  1.40s/it]



Train or Test set: Average loss: 0.0553, Accuracy: 502/512 (98.05%)


...


Train or Test set: Average loss: 0.0470, Accuracy: 506/512 (98.83%)


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 1.5510, Accuracy: 2750/4000 (68.75%)
Epoch:     5


100%|██████████| 465/465 [00:23<00:00, 20.11it/s]



Train or Test set: Average loss: 0.0129, Accuracy: 511/512 (99.80%)

Train or Test set: Average loss: 1.8565, Accuracy: 2506/4000 (62.65%)

Train or Test set: Average loss: 1.9327, Accuracy: 5008/7721 (64.86%)
New Best Accuracy: (64.86%) at Epoch: (5)

MY Current Accuracy: 64.86206449941717, at Epoch: 1
Post Training Quantization: Convert done

Train or Test set: Average loss: 1.9327, Accuracy: 5008/7721 (64.86%)
Size Prev (MB): 0.830143, Size Current (MB): 0.300794, Comp Rate: 2.7598389595537145 
[[87.82451923 66.50692915  2.75986476]
 [86.91105769 72.20567284  2.75984641]
 [89.66346154 65.53555239  2.75982141]
 [90.66105769 66.80481803  2.75984641]
 [88.26923077 64.8620645   2.75983896]]
Accuracy: 67.1830073824634, Acc_Std: 2.604924608476671, Comp Rate Avg.: 2.7598435877761665, Comp Rate Std.: 1.3981596461073544e-05


In [14]:
# ADAM results: LR=0.0005
# outer loop: 4 epochs
# inner loop: 5 epochs
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,64,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.3260, Accuracy: 7543/8320 (90.66%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [00:39<00:00, 11.82it/s]



Train or Test set: Average loss: 0.0516, Accuracy: 504/512 (98.44%)


...


Train or Test set: Average loss: 0.0101, Accuracy: 511/512 (99.80%)


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 1.6675, Accuracy: 2400/4000 (60.00%)
Epoch:     5


100%|██████████| 465/465 [00:40<00:00, 11.62it/s]



Train or Test set: Average loss: 0.0107, Accuracy: 511/512 (99.80%)

Train or Test set: Average loss: 1.9524, Accuracy: 2430/4000 (60.75%)

Train or Test set: Average loss: 1.9397, Accuracy: 4860/7721 (62.95%)
New Best Accuracy: (62.95%) at Epoch: (5)

MY Current Accuracy: 62.94521435047274, at Epoch: 3
Post Training Quantization: Convert done

Train or Test set: Average loss: 1.9397, Accuracy: 4860/7721 (62.95%)
Size Prev (MB): 0.830148, Size Current (MB): 0.300795, Comp Rate: 2.7598464070213935 
[[90.66105769 72.1279627   2.75989641]
 [88.26923077 66.10542676  2.75983471]
 [78.17307692 62.94521435  2.75984641]]
Accuracy: 67.05953460259896, Acc_Std: 3.8090644645609486, Comp Rate Avg.: 2.7598591730981945, Comp Rate Std.: 2.6757838887516586e-05


In [11]:
# ADAM results
# outer loop: 3 epochs
# inner loop: 5 epochs
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,64,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.3260, Accuracy: 7543/8320 (90.66%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [00:41<00:00, 11.11it/s]



Train or Test set: Average loss: 0.1107, Accuracy: 495/512 (96.68%)


...


Train or Test set: Average loss: 0.0079, Accuracy: 511/512 (99.80%)


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 2.0184, Accuracy: 2364/4000 (59.10%)
Epoch:    10


100%|██████████| 465/465 [00:47<00:00,  9.89it/s]



Train or Test set: Average loss: 0.0214, Accuracy: 508/512 (99.22%)

Train or Test set: Average loss: 1.8898, Accuracy: 2555/4000 (63.88%)

Train or Test set: Average loss: 2.0159, Accuracy: 4983/7721 (64.54%)
New Best Accuracy: (64.54%) at Epoch: (10)

MY Current Accuracy: 64.53827224452792, at Epoch: 2
Post Training Quantization: Convert done

Train or Test set: Average loss: 2.0159, Accuracy: 4983/7721 (64.54%)
Size Prev (MB): 0.830148, Size Current (MB): 0.30079, Comp Rate: 2.759892283653047 
[[90.66105769 61.92203083  2.75988311]
 [88.26923077 66.48102577  2.75983471]
 [78.17307692 64.53827224  2.75989228]]
Accuracy: 64.31377628113803, Acc_Std: 1.867959236541813, Comp Rate Avg.: 2.7598700325520036, Comp Rate Std.: 2.5259082333184534e-05


In [9]:
# ADAM results
# outer loop: 3 epochs
# inner loop: 5 epochs
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,64,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.4772, Accuracy: 7307/8320 (87.82%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [00:32<00:00, 14.31it/s]



Train or Test set: Average loss: 0.0614, Accuracy: 498/512 (97.27%)


...


Train or Test set: Average loss: 0.0151, Accuracy: 509/512 (99.41%)


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 1.4692, Accuracy: 2659/4000 (66.47%)
Epoch:    10


100%|██████████| 465/465 [00:40<00:00, 11.45it/s]



Train or Test set: Average loss: 0.0162, Accuracy: 509/512 (99.41%)

Train or Test set: Average loss: 1.4986, Accuracy: 2965/4000 (74.12%)

Train or Test set: Average loss: 1.5797, Accuracy: 5671/7721 (73.45%)
New Best Accuracy: (73.45%) at Epoch: (10)

MY Current Accuracy: 73.44903509908043, at Epoch: 2
Post Training Quantization: Convert done

Train or Test set: Average loss: 1.5797, Accuracy: 5671/7721 (73.45%)
Size Prev (MB): 0.830142, Size Current (MB): 0.300795, Comp Rate: 2.7598264598813147 
[[87.82451923 57.71273151  2.75986968]
 [86.91105769 72.46470664  2.75982806]
 [89.66346154 73.4490351   2.75982646]]
Accuracy: 67.87549108491991, Acc_Std: 7.197383192878395, Comp Rate Avg.: 2.7598413984539705, Comp Rate Std.: 2.0007767267647668e-05


In [7]:
# ADAM results
# outer loop: 3 epochs
# inner loop: 5 epochs
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,32,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.5529, Accuracy: 3659/4224 (86.62%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [00:22<00:00, 20.35it/s]



Train or Test set: Average loss: 0.0485, Accuracy: 504/512 (98.44%)


...


Train or Test set: Average loss: 0.0108, Accuracy: 510/512 (99.61%)


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 1.6725, Accuracy: 2395/4000 (59.88%)
Epoch:     5


100%|██████████| 465/465 [00:33<00:00, 13.76it/s]



Train or Test set: Average loss: 0.0131, Accuracy: 511/512 (99.80%)

Train or Test set: Average loss: 1.2396, Accuracy: 2652/4000 (66.30%)

Train or Test set: Average loss: 1.6001, Accuracy: 4956/7721 (64.19%)
New Best Accuracy: (64.19%) at Epoch: (5)

MY Current Accuracy: 64.1885766092475, at Epoch: 2
Post Training Quantization: Convert done

Train or Test set: Average loss: 1.6001, Accuracy: 4956/7721 (64.19%)
Size Prev (MB): 0.83015, Size Current (MB): 0.300795, Comp Rate: 2.7598530560680867 
[[86.62405303 68.57919959  2.75982473]
 [83.73579545 65.82048957  2.75983471]
 [91.45359848 64.18857661  2.75985306]]
Accuracy: 66.19608858956093, Acc_Std: 1.8120335713280693, Comp Rate Avg.: 2.759837498053629, Comp Rate Std.: 1.1730488894010597e-05


In [15]:
# ADAM results
# outer loop: 3 epochs
# inner loop: 5 epochs
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,32,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.5529, Accuracy: 3659/4224 (86.62%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [00:44<00:00, 10.47it/s]



Train or Test set: Average loss: 0.0484, Accuracy: 503/512 (98.24%)


...


Train or Test set: Average loss: 0.0104, Accuracy: 512/512 (100.00%)


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 1.6580, Accuracy: 2502/4000 (62.55%)
Epoch:     5


100%|██████████| 465/465 [00:40<00:00, 11.46it/s]



Train or Test set: Average loss: 0.0091, Accuracy: 511/512 (99.80%)

Train or Test set: Average loss: 1.9217, Accuracy: 2554/4000 (63.85%)

Train or Test set: Average loss: 1.7055, Accuracy: 5071/7721 (65.68%)
New Best Accuracy: (65.68%) at Epoch: (5)

MY Current Accuracy: 65.67802098173811, at Epoch: 2
Post Training Quantization: Convert done

Train or Test set: Average loss: 1.7055, Accuracy: 5071/7721 (65.68%)
Size Prev (MB): 0.830151, Size Current (MB): 0.300796, Comp Rate: 2.7598472054149656 
[[86.62405303 66.93433493  2.75985638]
 [83.73579545 65.74277943  2.75984721]
 [91.45359848 65.67802098  2.75984721]]
Accuracy: 66.11837844838752, Acc_Std: 0.5775737471090844, Comp Rate Avg.: 2.7598502638071216, Comp Rate Std.: 4.32521966561606e-06


In [13]:
# ADAM results
# outer loop: 8 epochs
# inner loop: 1 epochs
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,32,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.5529, Accuracy: 3659/4224 (86.62%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [00:39<00:00, 11.77it/s]



Train or Test set: Average loss: 0.0295, Accuracy: 507/512 (99.02%)


...


Train or Test set: Average loss: 0.0075, Accuracy: 511/512 (99.80%)


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 1.9090, Accuracy: 2540/4000 (63.50%)
Epoch:     5


100%|██████████| 465/465 [00:25<00:00, 18.22it/s]



Train or Test set: Average loss: 0.0039, Accuracy: 511/512 (99.80%)

Train or Test set: Average loss: 1.4924, Accuracy: 2570/4000 (64.25%)

Train or Test set: Average loss: 1.7955, Accuracy: 5097/7721 (66.01%)
New Best Accuracy: (66.01%) at Epoch: (5)

MY Current Accuracy: 66.01476492682295, at Epoch: 7
Post Training Quantization: Convert done

Train or Test set: Average loss: 1.7955, Accuracy: 5097/7721 (66.01%)
Size Prev (MB): 0.830151, Size Current (MB): 0.300796, Comp Rate: 2.7598472054149656 
[[86.62405303 64.43465872  2.75989135]
 [83.73579545 66.45512239  2.75983138]
 [91.45359848 66.01476493  2.75984721]]
Accuracy: 65.63484868108621, Acc_Std: 0.8674947170992638, Comp Rate Avg.: 2.759856646855544, Comp Rate Std.: 2.5377582195148056e-05


In [8]:
# ADAM results
# outer loop: 8 epochs
# inner loop: 5 epochs
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,32,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.5529, Accuracy: 3659/4224 (86.62%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [00:52<00:00,  8.81it/s]



Train or Test set: Average loss: 0.0533, Accuracy: 498/512 (97.27%)


...


Train or Test set: Average loss: 0.0029, Accuracy: 512/512 (100.00%)


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 1.7942, Accuracy: 2468/4000 (61.70%)
Epoch:     5


100%|██████████| 465/465 [00:23<00:00, 19.52it/s]



Train or Test set: Average loss: 0.0042, Accuracy: 512/512 (100.00%)

Train or Test set: Average loss: 1.8447, Accuracy: 2387/4000 (59.67%)

Train or Test set: Average loss: 2.1493, Accuracy: 4923/7721 (63.76%)
New Best Accuracy: (63.76%) at Epoch: (5)

MY Current Accuracy: 63.76117083279368, at Epoch: 7
Post Training Quantization: Convert done

Train or Test set: Average loss: 2.1493, Accuracy: 4923/7721 (63.76%)
Size Prev (MB): 0.830152, Size Current (MB): 0.300795, Comp Rate: 2.7598597051147795 
[[86.62405303 72.01139749  2.75981808]
 [83.73579545 66.89547986  2.75984135]
 [91.45359848 63.76117083  2.75985971]]
Accuracy: 67.55601606009584, Acc_Std: 3.4003716479806787, Comp Rate Avg.: 2.7598397144025264, Comp Rate Std.: 1.7031582652680755e-05


In [ ]:
# ADAM results
getResult4qaQuantChannel(args,stl10,train_loader_stl10,test_loader_stl10,32,1000)

In [10]:
# ADAM results
getResult4qaQuantChannel(args,urbansound,train_loader_urbansound,test_loader_urbansound,32,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/62 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.2808, Accuracy: 3866/4224 (91.52%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 62/62 [00:05<00:00, 10.39it/s]



Train or Test set: Average loss: 0.0267, Accuracy: 509/512 (99.41%)


...


Train or Test set: Average loss: 0.0095, Accuracy: 512/512 (100.00%)


  0%|          | 0/62 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.3613, Accuracy: 791/873 (90.61%)
Epoch:     5


100%|██████████| 62/62 [00:04<00:00, 15.06it/s]



Train or Test set: Average loss: 0.0030, Accuracy: 512/512 (100.00%)

Train or Test set: Average loss: 0.3359, Accuracy: 799/873 (91.52%)

Train or Test set: Average loss: 0.3324, Accuracy: 794/873 (90.95%)
MY Current Accuracy: 90.95074455899199, at Epoch: 7
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.3324, Accuracy: 794/873 (90.95%)
Size Prev (MB): 0.556102, Size Current (MB): 0.209748, Comp Rate: 2.6512863054713276 
[[91.52462121 89.91981672  2.65125626]
 [88.51799242 92.21076747  2.65129274]
 [88.80208333 90.95074456  2.65128631]]
Accuracy: 91.02710958381061, Acc_Std: 0.9368342227098378, Comp Rate Avg.: 2.651278432841234, Comp Rate Std.: 1.5898588040460204e-05


In [24]:
# ADAM results
getResult4qaQuantChannel(args,urbansound,train_loader_urbansound,test_loader_urbansound,32,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/62 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.3010, Accuracy: 3858/4224 (91.34%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 62/62 [00:03<00:00, 17.16it/s]



Train or Test set: Average loss: 0.0527, Accuracy: 504/512 (98.44%)

Train or Test set: Average loss: 0.3938, Accuracy: 778/873 (89.12%)
New Best Accuracy: (89.12%) at Epoch: (1)



...


Train or Test set: Average loss: 0.0240, Accuracy: 509/512 (99.41%)

Train or Test set: Average loss: 0.3522, Accuracy: 790/873 (90.49%)
New Best Accuracy: (90.49%) at Epoch: (1)



  0%|          | 0/62 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.3543, Accuracy: 793/873 (90.84%)
New Best Accuracy: (90.84%) at Epoch: (1)

MY Current Accuracy: 90.83619702176404, at Epoch: 6
Epoch:     1


100%|██████████| 62/62 [00:03<00:00, 20.42it/s]



Train or Test set: Average loss: 0.0124, Accuracy: 511/512 (99.80%)

Train or Test set: Average loss: 0.3548, Accuracy: 787/873 (90.15%)
New Best Accuracy: (90.15%) at Epoch: (1)


Train or Test set: Average loss: 0.3712, Accuracy: 772/873 (88.43%)
MY Current Accuracy: 88.43069873997709, at Epoch: 7
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.3712, Accuracy: 772/873 (88.43%)
Size Prev (MB): 0.556103, Size Current (MB): 0.209747, Comp Rate: 2.6513037135215285 
[[91.33522727 91.63802978  2.65130371]
 [85.65340909 88.43069874  2.65130371]]
Accuracy: 90.03436426116838, Acc_Std: 1.6036655211912958, Comp Rate Avg.: 2.6513037135215285, Comp Rate Std.: 0.0


In [11]:
# ADAM results
getResult4qaQuantChannel(args,pamap2,train_loader_pamap2,test_loader_pamap2,32,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/276 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.0459, Accuracy: 4169/4224 (98.70%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 276/276 [00:24<00:00, 11.26it/s]



Train or Test set: Average loss: 0.0247, Accuracy: 508/512 (99.22%)


...


Train or Test set: Average loss: 0.0017, Accuracy: 512/512 (100.00%)


  0%|          | 0/276 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.5950, Accuracy: 3489/4000 (87.22%)
Epoch:     5


100%|██████████| 276/276 [00:14<00:00, 18.86it/s]



Train or Test set: Average loss: 0.0015, Accuracy: 512/512 (100.00%)

Train or Test set: Average loss: 0.4851, Accuracy: 3549/4000 (88.72%)

Train or Test set: Average loss: 0.4110, Accuracy: 4736/5209 (90.92%)
New Best Accuracy: (90.92%) at Epoch: (5)

MY Current Accuracy: 90.91956229602611, at Epoch: 7
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.4110, Accuracy: 4736/5209 (90.92%)
Size Prev (MB): 0.827461, Size Current (MB): 0.299645, Comp Rate: 2.7614710741043567 
[[98.69791667 88.99980802  2.76147616]
 [95.76231061 90.13246304  2.76149618]
 [92.8030303  90.9195623   2.76147107]]
Accuracy: 90.01727778844308, Acc_Std: 0.7879572104034652, Comp Rate Avg.: 2.761481137061828, Comp Rate Std.: 1.0837766076749618e-05


In [25]:
# ADAM results
getResult4qaQuantChannel(args,pamap2,train_loader_pamap2,test_loader_pamap2,32,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/276 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.0657, Accuracy: 4136/4224 (97.92%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 276/276 [00:17<00:00, 15.37it/s]



Train or Test set: Average loss: 0.0205, Accuracy: 508/512 (99.22%)

Train or Test set: Average loss: 0.4134, Accuracy: 3510/4000 (87.75%)


...


Train or Test set: Average loss: 0.0030, Accuracy: 512/512 (100.00%)

Train or Test set: Average loss: 0.5509, Accuracy: 3524/4000 (88.10%)

Train or Test set: Average loss: 0.4393, Accuracy: 4671/5209 (89.67%)
New Best Accuracy: (89.67%) at Epoch: (1)

MY Current Accuracy: 89.6717220195815, at Epoch: 6


  0%|          | 0/276 [00:00<?, ?it/s]

Epoch:     1


100%|██████████| 276/276 [02:14<00:00,  2.05it/s]



Train or Test set: Average loss: 0.0046, Accuracy: 511/512 (99.80%)

Train or Test set: Average loss: 0.5069, Accuracy: 3495/4000 (87.38%)

Train or Test set: Average loss: 0.4395, Accuracy: 4657/5209 (89.40%)
New Best Accuracy: (89.40%) at Epoch: (1)

MY Current Accuracy: 89.40295642157804, at Epoch: 7
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.4395, Accuracy: 4657/5209 (89.40%)
Size Prev (MB): 0.827462, Size Current (MB): 0.299643, Comp Rate: 2.761492843150015 
[[97.91666667 88.78863505  2.76150126]
 [96.61458333 89.40295642  2.76149284]]
Accuracy: 89.09579573814551, Acc_Std: 0.3071606834325209, Comp Rate Avg.: 2.7614970531641427, Comp Rate Std.: 4.2100141273149205e-06


In [9]:
# ADAM results
getResult4qaQuantChannel(args,skoda,train_loader_skoda,test_loader_skoda,32,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/79 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.3534, Accuracy: 3778/4224 (89.44%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 79/79 [00:05<00:00, 13.64it/s]



Train or Test set: Average loss: 0.0864, Accuracy: 495/512 (96.68%)


...


Train or Test set: Average loss: 0.0025, Accuracy: 512/512 (100.00%)


  0%|          | 0/79 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.1210, Accuracy: 1163/1193 (97.49%)
Epoch:     5


100%|██████████| 79/79 [00:03<00:00, 24.58it/s]



Train or Test set: Average loss: 0.0067, Accuracy: 510/512 (99.61%)

Train or Test set: Average loss: 0.1183, Accuracy: 1164/1193 (97.57%)

Train or Test set: Average loss: 0.1067, Accuracy: 1164/1193 (97.57%)
MY Current Accuracy: 97.56915339480302, at Epoch: 7
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.1067, Accuracy: 1164/1193 (97.57%)
Size Prev (MB): 0.777415, Size Current (MB): 0.284154, Comp Rate: 2.7358932128352933 
[[89.44128788 97.4015088   2.73586599]
 [95.59659091 96.73093043  2.73587748]
 [98.67424242 97.56915339  2.73589321]]
Accuracy: 97.23386420787931, Acc_Std: 0.36215371323877316, Comp Rate Avg.: 2.7358788924070048, Comp Rate Std.: 1.1159285522283347e-05


In [26]:
# ADAM results
getResult4qaQuantChannel(args,skoda,train_loader_skoda,test_loader_skoda,32,1000)

QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/79 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.3803, Accuracy: 3754/4224 (88.87%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 79/79 [00:33<00:00,  2.33it/s]



Train or Test set: Average loss: 0.0533, Accuracy: 503/512 (98.24%)

Train or Test set: Average loss: 0.2078, Accuracy: 1134/1193 (95.05%)
New Best Accuracy: (95.05%) at Epoch: (1)


Train or Test set: Average loss: 0.1773, Accuracy: 1131/1193 (94.80%)
MY Current Accuracy: 94.80301760268232, at Epoch: 0


...


Train or Test set: Average loss: 0.0160, Accuracy: 508/512 (99.22%)

Train or Test set: Average loss: 0.1225, Accuracy: 1156/1193 (96.90%)
New Best Accuracy: (96.90%) at Epoch: (1)


Train or Test set: Average loss: 0.1140, Accuracy: 1161/1193 (97.32%)
New Best Accuracy: (97.32%) at Epoch: (1)

MY Current Accuracy: 97.31768650461022, at Epoch: 6


  0%|          | 0/79 [00:00<?, ?it/s]

Epoch:     1


100%|██████████| 79/79 [00:11<00:00,  6.94it/s]



Train or Test set: Average loss: 0.0173, Accuracy: 507/512 (99.02%)

Train or Test set: Average loss: 0.1202, Accuracy: 1163/1193 (97.49%)
New Best Accuracy: (97.49%) at Epoch: (1)


Train or Test set: Average loss: 0.1365, Accuracy: 1154/1193 (96.73%)
MY Current Accuracy: 96.73093042749372, at Epoch: 7
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.1365, Accuracy: 1154/1193 (96.73%)
Size Prev (MB): 0.777417, Size Current (MB): 0.284153, Comp Rate: 2.7359098795367287 
[[88.87310606 97.4853311   2.73584839]
 [91.21685606 96.73093043  2.73590988]]
Accuracy: 97.1081307627829, Acc_Std: 0.3772003352891815, Comp Rate Avg.: 2.735879136069679, Comp Rate Std.: 3.07434670498985e-05
